In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
# from tqdm.notebook import tqdm

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.svm import LinearSVC, SVC, OneClassSVM

In [4]:
file = pd.read_csv(r'../Dataset/Dataset_13.05.2020_11.18am_1.csv')
tweets = np.asarray(file['Tweet'].copy())
targets = np.asarray(file['Is_Unreliable'].copy())
tweet_embeddings = np.load(r'../Dataset/Tweet_Embeddings_Word2Vec_13.05.2020_2.41pm_1.npy', allow_pickle=True)
# tweet_embeddings = np.load(r'../Dataset/Tweet_Embeddings_Glove200_lowercase_22.05.2020_8.35pm_1.npy', allow_pickle=True)
# tweet_embeddings = np.load(r'../Dataset/Tweet_Embeddings_Glove100_lowercase_22.05.2020_8.32pm_1.npy', allow_pickle=True)
# tweet_embeddings = np.load(r'../Dataset/Tweet_Embeddings_Glove50_lowercase_22.05.2020_8.31pm_1.npy', allow_pickle=True)
# tweet_embeddings = np.load(r'../Dataset/Tweet_Embeddings_Glove25_lowercase_22.05.2020_8.30pm_1.npy', allow_pickle=True)
print(tweet_embeddings.shape)
print(tweet_embeddings[0].shape)
X = list()
for tweet in tweet_embeddings:
#     embeddings = np.mean(tweet, axis=0, dtype=np.float64)
    embeddings = np.amax(tweet.astype(np.float64), axis=0)
    X.append(embeddings)
X = np.asarray(X)
print(X.shape)
print(X.dtype)

y = targets
print(y.shape)
print(y.dtype)

(560,)
(15, 300)
(560, 300)
float64
(560,)
int64


In [12]:
class AE(nn.Module):

    def __init__(self, hidden_size=100):
        super(AE, self).__init__()
        self.fc1 = nn.Linear(300, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 300)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# net = AE()
# print(net)

def train_AE(model, X_train, y_train, epochs=100, to_print=False, print_interval=10):
    # define the optimization
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    X_train = torch.from_numpy(X_train).float()
    train_mse_list = list()
    
    for epoch in tqdm(range(epochs)):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        train_loss = criterion(output, X_train)
        train_loss.backward()
        optimizer.step()
        if to_print and epoch%print_interval == (print_interval-1):
            print("epoch : {}/{}, train_loss = {:.6f}".format(epoch + 1, epochs, train_loss))
        train_mse_list.append(train_loss)
        
    return model, train_mse_list

def train_AE_wo_tqdm(model, X_train, y_train, epochs=100, to_print=False, print_interval=10):
    # define the optimization
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    X_train = torch.from_numpy(X_train).float()
    train_mse_list = list()
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        train_loss = criterion(output, X_train)
        train_loss.backward()
        optimizer.step()
        if to_print and epoch%print_interval == (print_interval-1):
            print("epoch : {}/{}, train_loss = {:.6f}".format(epoch + 1, epochs, train_loss))
        train_mse_list.append(train_loss.item())
        
    return model, train_mse_list

def predict_AE(model, X_train, X_test, threshold, above_thres_class=0):
    criterion = nn.MSELoss(reduction='none')
    
    model.eval()
    with torch.no_grad():
        X_train = torch.from_numpy(X_train).float()
        X_test = torch.from_numpy(X_test).float()
        train_out = model(X_train)
        test_out = model(X_test)
        train_mse = torch.mean(criterion(train_out, X_train), axis=1)
        test_mse = torch.mean(criterion(test_out, X_test), axis=1)
        
    train_mse, test_mse = train_mse.numpy(), test_mse.numpy()
    train_preds, test_preds = np.empty((train_mse.shape[0])), np.empty((test_mse.shape[0]))
    train_preds[train_mse > threshold] = above_thres_class
    train_preds[train_mse <= threshold] = (above_thres_class+1)%2
    test_preds[test_mse > threshold] = above_thres_class
    test_preds[test_mse <= threshold] = (above_thres_class+1)%2
    
    return (train_preds, test_preds)

def predict_AE_for_ROC(model, X_train, X_test):
    criterion = nn.MSELoss(reduction='none')
    
    model.eval()
    with torch.no_grad():
        X_train = torch.from_numpy(X_train).float()
        X_test = torch.from_numpy(X_test).float()
        train_out = model(X_train)
        test_out = model(X_test)
        train_mse = torch.mean(criterion(train_out, X_train), axis=1)
        test_mse = torch.mean(criterion(test_out, X_test), axis=1)
    
    train_mse, test_mse = train_mse.numpy(), test_mse.numpy()
    
    return (train_mse, test_mse)

In [6]:
percent = 0.9

X_0, X_1_, y_0, y_1_ = X[y==0].copy(), X[y==1].copy(), y[y==0].copy(), y[y==1].copy()
X_1, _, y_1, _ = train_test_split(X_1_, y_1_, train_size=percent, shuffle=True, random_state=0)

X_train, X_test_, y_train, y_test_ = train_test_split(X_0, y_0, train_size = 0.67, shuffle=True, random_state=0)
X_test, y_test = np.concatenate((X_1.copy(), X_test_.copy())), np.concatenate((y_1.copy(), y_test_.copy()))
X_test, y_test = shuffle(X_test, y_test, random_state=0)

# y_train[y_train == 1] = -1
# y_train[y_train == 0] = 1
# y_train[y_train == -1] = 0
# y_test[y_test == 1] = -1
# y_test[y_test == 0] = 1
# y_test[y_test == -1] = 0

print(X_train.shape, y_train.shape, sum(y_train))
print(X_test.shape, y_test.shape, sum(y_test))

(187, 300) (187,) 0
(345, 300) (345,) 252


In [21]:
# model = AE()
# criterion = nn.MSELoss(reduction='none')

# with torch.no_grad():
#     loss = torch.mean(criterion(model(torch.from_numpy(X_train).float()), torch.from_numpy(X_train).float()), axis=1)
#     print(loss.shape)

torch.Size([187])


In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AE(hidden_size=100).to(device)
# print(model)
n_epochs = 100
for noise_percent in (0.1, 0.2, 0.3, 0.4, 0.5):
    number_of_copies = 5
    noisy_data = np.concatenate(([X_train]*number_of_copies), axis=0)
#     print(noisy_data.shape)
#     np.random.seed(None)
    for i in np.arange(noisy_data.shape[0]):
        idx = np.random.permutation(np.arange(noisy_data.shape[1]))[0:int(noise_percent*noisy_data.shape[1])]
        noisy_data[i][idx] = 0
    
    model, _ = train_AE(model, noisy_data, y_train, to_print=False, epochs=n_epochs, print_interval=n_epochs/10)
    best_f = 0
    train_preds, test_preds = np.empty((X_train.shape[0])), np.empty((X_test.shape[0]))
    for th in np.arange(0.0001, 0.0100, 0.0001):
        (tr_preds, te_preds) = predict_AE(model, X_train, X_test, threshold=th, above_thres_class=0)
        if (f1_score(y_test, te_preds, average='weighted') > best_f):
            best_f = f1_score(y_test, te_preds, average='weighted')
            train_preds = tr_preds
            test_preds = te_preds

    print('Acc:', accuracy_score(y_test, test_preds))
    print('F1 :', f1_score(y_test, test_preds, average='weighted'))
    print('P  :', precision_score(y_test, test_preds, average='weighted'))
    print('R  :', recall_score(y_test, test_preds, average='weighted'))
    # print('ROC:', roc_auc_score(y_test, test_preds)) #Should be used with probability values
    print('\nConfusion Matrix')
    print(confusion_matrix(y_test, test_preds))

 13%|█▎        | 13/100 [00:00<00:00, 125.55it/s]

Acc: 0.6811594202898551
F1 : 0.6800567107750473
P  : 0.6789974831741349
R  : 0.6811594202898551

Confusion Matrix
[[ 37  56]
 [ 54 198]]


 11%|█         | 11/100 [00:00<00:00, 102.93it/s]

Acc: 0.7130434782608696
F1 : 0.6790130571268255
P  : 0.671294466403162
R  : 0.7130434782608696

Confusion Matrix
[[ 21  72]
 [ 27 225]]


  9%|▉         | 9/100 [00:00<00:01, 85.88it/s]

Acc: 0.7043478260869566
F1 : 0.6806236335950593
P  : 0.6709875238983566
R  : 0.7043478260869566

Confusion Matrix
[[ 25  68]
 [ 34 218]]


  7%|▋         | 7/100 [00:00<00:01, 68.09it/s]

Acc: 0.6898550724637681
F1 : 0.6859150664967509
P  : 0.6825239799961741
R  : 0.6898550724637681

Confusion Matrix
[[ 36  57]
 [ 50 202]]


100%|██████████| 100/100 [00:00<00:00, 113.78it/s]


Acc: 0.7014492753623188
F1 : 0.6802438033319788
P  : 0.6705484713473652
R  : 0.7014492753623188

Confusion Matrix
[[ 26  67]
 [ 36 216]]


In [60]:
# noisy_data = np.concatenate(([X_train]*number_of_copies), axis=0)
# print(noisy_data[0][0])
# print(X_train[0][0])
# noisy_data[0][0] = 0
# print(noisy_data[0][0])
# print(X_train[0][0])

0.019091796875
0.019091796875
0.0
0.019091796875


In [ ]:
# # Autoencoder and OC-SVM on class=0 as training data (class=0 has NOT been changed to class=1)

# for percent in np.arange(0.9, 0.01, -0.1):
#     X_0, X_1_, y_0, y_1_ = X[y==0].copy(), X[y==1].copy(), y[y==0].copy(), y[y==1].copy()
#     X_1, _, y_1, _ = train_test_split(X_1_, y_1_, train_size=percent, shuffle=True, random_state=0)

#     X_train, X_test_, y_train, y_test_ = train_test_split(X_0, y_0, train_size = 0.67, shuffle=True, random_state=0)
#     X_test, y_test = np.concatenate((X_1.copy(), X_test_.copy())), np.concatenate((y_1.copy(), y_test_.copy()))
#     X_test, y_test = shuffle(X_test, y_test, random_state=0)
    
#     clf = OneClassSVM(nu=len(y_1)/len(y_0))
#     best_roc_auc = 0
#     for kl in tqdm(('linear', 'poly', 'rbf', 'sigmoid')):
#         for gm in ('auto', 'scale', 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001):
#             this_clf = OneClassSVM(kernel=kl, gamma=gm, nu=len(y_1)/len(y_0))
#             this_clf.fit(X_train)
#             clf_test_vals = this_clf.decision_function(X_test)
#             roc_auc_till_now = roc_auc_score(y_test, (max(clf_test_vals) - clf_test_vals), average='macro')
#             if ((roc_auc_till_now > best_roc_auc) and (this_clf.fit_status_ == 0)):
#                 best_roc_auc = roc_auc_till_now
#                 clf = this_clf
                
            
#     clf.fit(X_train)
#     c_train_preds = clf.predict(X_train)
#     c_test_preds = clf.predict(X_test)

#     c_train_preds[c_train_preds == 1] = 0
#     c_train_preds[c_train_preds == -1] = 1
#     c_test_preds[c_test_preds == 1] = 0
#     c_test_preds[c_test_preds == -1] = 1

#     # c_train_preds[c_train_preds == -1] = 0
#     # c_test_preds[c_test_preds == -1] = 0

#     print('Fit status:', clf.fit_status_) # 0 if correctly fitted 
#     print('')
#     print('Acc:', accuracy_score(y_test, c_test_preds))
# #     print('F1 :', f1_score(y_test, c_test_preds, average='macro'))
# #     print('P  :', precision_score(y_test, c_test_preds, average='macro'))
# #     print('R  :', recall_score(y_test, c_test_preds, average='macro'))
#     print('F1 :', f1_score(y_test, c_test_preds, average=None))
#     print('P  :', precision_score(y_test, c_test_preds, average=None))
#     print('R  :', recall_score(y_test, c_test_preds, average=None))
#     # print('ROC:', roc_auc_score(y_test, c_test_preds, average='macro')) #Should be used with probability values
#     print('\nConfusion Matrix')
#     print(confusion_matrix(y_test, c_test_preds))
    

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = AE(hidden_size=100).to(device)
# #     print(model)
    
#     number_of_copies = 1 # copies of X_train for noisy_data
    
#     best_roc_auc = 0
#     for h_size in tqdm(np.arange(5, 300, 20)):
#         for n_epochs in np.arange(10, 101, 20):
#             # this_model = AE(hidden_size=h_size).to(device)
#             for noise_percent in (0.1, 0.2, 0.3, 0.4, 0.5):
# #                 number_of_copies = 5
#                 noisy_data = np.concatenate(([X_train]*number_of_copies), axis=0)
#             #     print(noisy_data.shape)
#             #     np.random.seed(None)
#                 for i in np.arange(noisy_data.shape[0]):
#                     idx = np.random.permutation(np.arange(noisy_data.shape[1]))[0:int(noise_percent*noisy_data.shape[1])]
#                     noisy_data[i][idx] = 0
                
#                 this_model = AE(hidden_size=h_size).to(device)
#                 this_model, _ = train_AE_wo_tqdm(this_model, noisy_data, y_train, to_print=False, epochs=n_epochs, print_interval=n_epochs/10)
#                 (_, test_mse) = predict_AE_for_ROC(this_model, X_train, X_test)
#                 roc_auc_till_now = roc_auc_score(y_test, test_mse, average='macro')
#                 if (roc_auc_till_now > best_roc_auc):
#                     model = this_model
            
#     print(model)
#     best_f = 0
#     train_preds, test_preds = np.empty((X_train.shape[0])), np.empty((X_test.shape[0]))
#     for th in np.arange(0.0001, 0.1000, 0.0001):
#         (tr_preds, te_preds) = predict_AE(model, X_train, X_test, threshold=th, above_thres_class=1)
#         if (f1_score(y_test, te_preds, average='macro') >= best_f):
#             best_f = f1_score(y_test, te_preds, average='macro')
#             train_preds = tr_preds
#             test_preds = te_preds
    
#     print('Acc:', accuracy_score(y_test, test_preds))
# #     print('F1 :', f1_score(y_test, test_preds, average='macro'))
# #     print('P  :', precision_score(y_test, test_preds, average='macro'))
# #     print('R  :', recall_score(y_test, test_preds, average='macro'))
#     print('F1 :', f1_score(y_test, test_preds, average=None))
#     print('P  :', precision_score(y_test, test_preds, average=None))
#     print('R  :', recall_score(y_test, test_preds, average=None))
#     # print('ROC:', roc_auc_score(y_test, test_preds, average='macro')) #Should be used with probability values
#     print('\nConfusion Matrix')
#     print(confusion_matrix(y_test, test_preds))
    
#     (train_mse, test_mse) = predict_AE_for_ROC(model, X_train, X_test)
#     fpr_AE, tpr_AE, _ = roc_curve(y_test, test_mse, pos_label=1)
#     roc_auc_AE = auc(fpr_AE, tpr_AE)
#     # fpr_AE_t, tpr_AE_t, thresholds_AE_t = roc_curve(y_train, train_mse, pos_label=1)
#     # roc_auc_AE_t = auc(fpr_AE_t, tpr_AE_t)
#     # print(tpr_AE)
#     # print(fpr_AE)
#     # print(thresholds_AE)

#     clf_train_vals, clf_test_vals = clf.decision_function(X_train), clf.decision_function(X_test)
#     fpr_c, tpr_c, _ = roc_curve(y_test, (max(clf_test_vals) - clf_test_vals), pos_label=1)
#     roc_auc_c = auc(fpr_c, tpr_c)
#     # fpr_c_t, tpr_c_t, thresholds_c_t = roc_curve(y_train, (max(clf_train_vals) - clf_train_vals), pos_label=1)
#     # roc_auc_c_t = auc(fpr_c_t, tpr_c_t)
#     # print(tpr_c)
#     # print(fpr_c)
#     # print(thresholds_c)

#     plt.figure()
#     plt.plot(fpr_AE, tpr_AE, color='darkorange', lw=2, label='ROC curve Test AE (area = %0.2f)' % roc_auc_AE)
#     plt.plot(fpr_c, tpr_c, color='brown', lw=2, label='ROC curve Test OC-SVM (area = %0.2f)' % roc_auc_c)
#     # plt.plot(fpr_AE_t, tpr_AE_t, color='yellow', lw=2, label='ROC curve Train AE (area = %0.2f)' % roc_auc_AE_t)
#     # plt.plot(fpr_c_t, tpr_c_t, color='pink', lw=2, label='ROC curve Train OC-SVM (area = %0.2f)' % roc_auc_c_t)
#     plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver Operating Characteristic for %0.2f' % percent)
#     plt.legend(loc="lower right")
#     plt.show()
    
#     lr_precision_AE, lr_recall_AE, _ = precision_recall_curve(y_test, test_mse, pos_label=1)
#     avg_p_AE = average_precision_score(y_test, test_mse, pos_label=1, average='macro')
#     lr_precision_c, lr_recall_c, _ = precision_recall_curve(y_test, (max(clf_test_vals) - clf_test_vals), pos_label=1)
#     avg_p_c = average_precision_score(y_test, (max(clf_test_vals) - clf_test_vals), pos_label=1, average='macro')
# #     lr_f1, lr_auc = f1_score(testy, yhat), auc(lr_recall, lr_precision)
# #     print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))

#     # plot the precision-recall curves
#     plt.figure()
#     plt.plot(lr_recall_AE, lr_precision_AE, color='darkorange', lw=2, label='PR curve Test AE (AP = %0.2f)' % avg_p_AE)
#     plt.plot(lr_recall_c, lr_precision_c, color='brown', lw=2, label='PR curve Test OC-SVM (AP = %0.2f)' % avg_p_c)
#     plt.plot([0, 1], [len(y_test[y_test==1])/len(y_test), len(y_test[y_test==1])/len(y_test)], color='navy', lw=2, linestyle='--')
#     plt.xlabel('Recall')
#     plt.ylabel('Precision')
#     plt.title('Precision-Recall Curve for %0.2f' % percent)
#     plt.legend()
#     plt.show()

In [ ]:
# Autoencoder and OC-SVM on class=0 as training data (class=0 has NOT been changed to class=1)

for percent in np.arange(0.9, 0.01, -0.1):
    X_0, X_1_, y_0, y_1_ = X[y==0].copy(), X[y==1].copy(), y[y==0].copy(), y[y==1].copy()
    X_1, _, y_1, _ = train_test_split(X_1_, y_1_, train_size=percent, shuffle=True, random_state=0)

    X_train, X_test_, y_train, y_test_ = train_test_split(X_0, y_0, train_size = 0.67, shuffle=True, random_state=0)
    X_test, y_test = np.concatenate((X_1.copy(), X_test_.copy())), np.concatenate((y_1.copy(), y_test_.copy()))
    X_test, y_test = shuffle(X_test, y_test, random_state=0)
    
    clf = OneClassSVM(nu=len(y_1)/len(y_0))
    best_roc_auc = 0
    for kl in tqdm(('linear', 'poly', 'rbf', 'sigmoid')):
        for gm in ('auto', 'scale', 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001):
            this_clf = OneClassSVM(kernel=kl, gamma=gm, nu=len(y_1)/len(y_0))
            this_clf.fit(X_train)
            clf_test_vals = this_clf.decision_function(X_test)
            roc_auc_till_now = roc_auc_score(y_test, (max(clf_test_vals) - clf_test_vals), average='macro')
            if ((roc_auc_till_now > best_roc_auc) and (this_clf.fit_status_ == 0)):
                best_roc_auc = roc_auc_till_now
                clf = this_clf
                
            
    clf.fit(X_train)
    c_train_preds = clf.predict(X_train)
    c_test_preds = clf.predict(X_test)

    c_train_preds[c_train_preds == 1] = 0
    c_train_preds[c_train_preds == -1] = 1
    c_test_preds[c_test_preds == 1] = 0
    c_test_preds[c_test_preds == -1] = 1

    # c_train_preds[c_train_preds == -1] = 0
    # c_test_preds[c_test_preds == -1] = 0

    print('Fit status:', clf.fit_status_) # 0 if correctly fitted 
    print('')
    print('Acc:', accuracy_score(y_test, c_test_preds))
#     print('F1 :', f1_score(y_test, c_test_preds, average='macro'))
#     print('P  :', precision_score(y_test, c_test_preds, average='macro'))
#     print('R  :', recall_score(y_test, c_test_preds, average='macro'))
    print('F1 :', f1_score(y_test, c_test_preds, average=None))
    print('P  :', precision_score(y_test, c_test_preds, average=None))
    print('R  :', recall_score(y_test, c_test_preds, average=None))
    # print('ROC:', roc_auc_score(y_test, c_test_preds, average='macro')) #Should be used with probability values
    print('\nConfusion Matrix')
    print(confusion_matrix(y_test, c_test_preds))
    

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AE(hidden_size=100).to(device)
#     print(model)
    
    number_of_copies = 5 # copies of X_train for noisy_data
    
    best_roc_auc = 0
    for h_size in tqdm(np.arange(5, 1000, 50)):
        for n_epochs in np.arange(10, 501, 50):
            # this_model = AE(hidden_size=h_size).to(device)
            noisy_data = np.concatenate(([X_train]*number_of_copies), axis=0)
            for j, noise_percent in enumerate((0.05, 0.10, 0.15, 0.20, 0.25)):
#                 number_of_copies = 5
                # noisy_data = np.concatenate(([X_train]*number_of_copies), axis=0)
            #     print(noisy_data.shape)
            #     np.random.seed(None)
                for i in np.arange(X_train.shape[0]):
                    idx = np.random.permutation(np.arange(noisy_data.shape[1]))[0:int(noise_percent*noisy_data.shape[1])]
                    noisy_data[X_train.shape[0]*j + i][idx] = 0
                
            this_model = AE(hidden_size=h_size).to(device)
            this_model, _ = train_AE_wo_tqdm(this_model, noisy_data, y_train, to_print=False, epochs=n_epochs, print_interval=n_epochs/10)
            (_, test_mse) = predict_AE_for_ROC(this_model, X_train, X_test)
            roc_auc_till_now = roc_auc_score(y_test, test_mse, average='macro')
            if (roc_auc_till_now > best_roc_auc):
                model = this_model
            
    print(model)
    best_f = 0
    train_preds, test_preds = np.empty((X_train.shape[0])), np.empty((X_test.shape[0]))
    for th in np.arange(0.0001, 0.1000, 0.0001):
        (tr_preds, te_preds) = predict_AE(model, X_train, X_test, threshold=th, above_thres_class=1)
        if (f1_score(y_test, te_preds, average='macro') >= best_f):
            best_f = f1_score(y_test, te_preds, average='macro')
            train_preds = tr_preds
            test_preds = te_preds
    
    print('Acc:', accuracy_score(y_test, test_preds))
#     print('F1 :', f1_score(y_test, test_preds, average='macro'))
#     print('P  :', precision_score(y_test, test_preds, average='macro'))
#     print('R  :', recall_score(y_test, test_preds, average='macro'))
    print('F1 :', f1_score(y_test, test_preds, average=None))
    print('P  :', precision_score(y_test, test_preds, average=None))
    print('R  :', recall_score(y_test, test_preds, average=None))
    # print('ROC:', roc_auc_score(y_test, test_preds, average='macro')) #Should be used with probability values
    print('\nConfusion Matrix')
    print(confusion_matrix(y_test, test_preds))
    
    (train_mse, test_mse) = predict_AE_for_ROC(model, X_train, X_test)
    fpr_AE, tpr_AE, thresholds_AE = roc_curve(y_test, test_mse, pos_label=1)
    roc_auc_AE = auc(fpr_AE, tpr_AE)
    # fpr_AE_t, tpr_AE_t, thresholds_AE_t = roc_curve(y_train, train_mse, pos_label=1)
    # roc_auc_AE_t = auc(fpr_AE_t, tpr_AE_t)
    # print(tpr_AE)
    # print(fpr_AE)
    # print(thresholds_AE)

    clf_train_vals, clf_test_vals = clf.decision_function(X_train), clf.decision_function(X_test)
    fpr_c, tpr_c, thresholds_c = roc_curve(y_test, (max(clf_test_vals) - clf_test_vals), pos_label=1)
    roc_auc_c = auc(fpr_c, tpr_c)
    # fpr_c_t, tpr_c_t, thresholds_c_t = roc_curve(y_train, (max(clf_train_vals) - clf_train_vals), pos_label=1)
    # roc_auc_c_t = auc(fpr_c_t, tpr_c_t)
    # print(tpr_c)
    # print(fpr_c)
    # print(thresholds_c)

    plt.figure()
    plt.plot(fpr_AE, tpr_AE, color='darkorange', lw=2, label='ROC curve Test AE (area = %0.2f)' % roc_auc_AE)
    plt.plot(fpr_c, tpr_c, color='brown', lw=2, label='ROC curve Test OC-SVM (area = %0.2f)' % roc_auc_c)
    # plt.plot(fpr_AE_t, tpr_AE_t, color='yellow', lw=2, label='ROC curve Train AE (area = %0.2f)' % roc_auc_AE_t)
    # plt.plot(fpr_c_t, tpr_c_t, color='pink', lw=2, label='ROC curve Train OC-SVM (area = %0.2f)' % roc_auc_c_t)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic for %0.2f' % percent)
    plt.legend(loc="lower right")
    plt.show()
    
    lr_precision_AE, lr_recall_AE, _ = precision_recall_curve(y_test, test_mse, pos_label=1)
    avg_p_AE = average_precision_score(y_test, test_mse, pos_label=1, average='macro')
    lr_precision_c, lr_recall_c, _ = precision_recall_curve(y_test, (max(clf_test_vals) - clf_test_vals), pos_label=1)
    avg_p_c = average_precision_score(y_test, (max(clf_test_vals) - clf_test_vals), pos_label=1, average='macro')
#     lr_f1, lr_auc = f1_score(testy, yhat), auc(lr_recall, lr_precision)
#     print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))

    # plot the precision-recall curves
    plt.figure()
    plt.plot(lr_recall_AE, lr_precision_AE, color='darkorange', lw=2, label='PR curve Test AE (AP = %0.2f)' % avg_p_AE)
    plt.plot(lr_recall_c, lr_precision_c, color='brown', lw=2, label='PR curve Test OC-SVM (AP = %0.2f)' % avg_p_c)
    plt.plot([0, 1], [len(y_test[y_test==1])/len(y_test), len(y_test[y_test==1])/len(y_test)], color='navy', lw=2, linestyle='--')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve for %0.2f' % percent)
    plt.legend()
    plt.show()